**ChatBot using Python**

**Data preparation from own created data_file**

In [28]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


**Reading JSON file ...**

In [29]:
with open('intents.JSON') as file:
  data=json.load(file)
  print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'], 'responses': ['Hello', 'Hi', 'Hi there', 'Greetings!', 'Hey, how can I help you today?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again', 'Goodbye! Take care.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!", 'Glad I could assist you.']}, {'tag': 'about', 'patterns': ['Who are you?', 'What are you?', 'Who you are?'], 'responses': ["I'm Anil, your bot assistant", 'I am Anil, an Artificial Intelligent bot', 'You can call me Anil.']}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Anil.', "I'm Anil!", 'Just call me as Anil', 'Anil is the name. How can I assist you today?']}, {'tag': 'help',

In [30]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
        responses.append(intent['responses'])
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

num_classes = len(labels)

label_encoder=LabelEncoder()
label_encoder.fit(training_labels)
training_labels=label_encoder.transform(training_labels)


**Tokenization of JSON file.**

In [32]:
vocab_size=2000
embedding_dim=16
max_len=20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

**Training a Neural Network**

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            32000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 13)                221       
                                                                 
Total params: 32,765
Trainable params: 32,765
Non-trainable params: 0
__________________________________________________

**Saving The Neural Network**

In [34]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

**Building a Chatbot with Python and Our Trained Machine Learning Model**

In [ ]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.JSON") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
1/1 [==============================] - 0s 80ms/step
ChatBot: Hello
User: what is personal interest?
1/1 [==============================] - 0s 28ms/step
ChatBot: Hey, how can I help you today?
User: what is your name?
1/1 [==============================] - 0s 28ms/step
ChatBot: You can call me Anil.
User: Tell me more about yourself?
1/1 [==============================] - 0s 31ms/step
ChatBot: I'm Anil, an AI language model created by OpenAI. I can assist you with a variety of tasks and provide information.
User: Any hidden talent?
1/1 [==============================] - 0s 36ms/step
ChatBot: Aside from answering questions, I can also help you with language-related tasks, trivia, and much more.
User: Are you goof about math?
1/1 [==============================] - 0s 27ms/step
ChatBot: My pleasure
User: what's your favorite branch of mathematics?
1/1 [==============================] - 0s 32ms/step
ChatBot: I am a computational sci